Descripción de los datos

Cada entrada de registro es una acción de usuario o un evento.

    EventName: nombre del evento.
    DeviceIDHash: identificador de usuario unívoco.
    EventTimestamp: hora del evento.
    ExpId: número de experimento. 246 y 247 son los grupos de control, y 248 es el grupo de prueba.

Instrucciones para completar el proyecto

Paso 1. Abrir el archivo de datos y leer la información general

Ruta de archivo: */datasets/logs_exp_us.csv* 


In [ ]:
# se importan las librerias necesarias para el analisis de datos
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import scipy.stats as stats
from matplotlib import pyplot as plt


def looks(datos):
    """Genera un análisis de todo el set de datos de manera global"""

    print("Se ejecuta la descripción de los datos:")
    print(datos.describe(include="all"))
    print()
    print("se hace exploración con método info:")
    print(datos.info())
    print()
    print("se hace una observación con el método Sample:")
    if len(datos) > 10:
        print(datos.sample(10))
    else:
        print(datos)
    print()
    print("hay NA?:")
    print(datos.isna().sum())
    print()
    print("Duplicados:")
    print(datos.duplicated().sum())

In [ ]:
data = pd.read_csv('logs_exp_us.csv' , sep='\t')
looks(data)

In [ ]:
#Se exploran los duplicados para ver si hay duplicados reales, Se determina que no, solo son distintos eventos para el mismo dispositivo, o el mismo evento para distintos dispositivos
print(data[data.duplicated()].sort_values('DeviceIDHash').head(15))


Paso 2. Preparar los datos para el análisis

    Cambia el nombre de las columnas de manera que sea conveniente para ti.
    Comprueba los tipos de datos y valores ausentes. Corrige los datos si es necesario.
    Agrega una columna de fecha y hora y una columna separada para las fechas.

In [ ]:
data['daytime'] = data['EventTimestamp'].apply(lambda x: dt.datetime.fromtimestamp(x))
data['date'] = data['daytime'].dt.date
data



Paso 3. Estudiar y comprobar los datos

  -  ¿Cuántos eventos hay en los registros?
  -  ¿Cuántos usuarios hay en los registros?
  -  ¿Cuál es el promedio de eventos por usuario?
  -  ¿Qué periodo de tiempo cubren los datos? Encuentra la fecha máxima y mínima. Traza un histograma por fecha y hora. ¿Puedes tener seguridad de que tienes datos igualmente completos para todo el periodo? Los eventos más antiguos podrían terminar en los registros de algunos usuarios por razones técnicas y esto podría sesgar el panorama general. Encuentra el momento en el que los datos comienzan a estar completos e ignora la sección anterior. ¿Qué periodo representan realmente los datos?
  -  ¿Perdiste muchos eventos y usuarios al excluir los datos más antiguos?
  -  Asegúrate de tener usuarios de los tres grupos experimentales.


In [ ]:
# ¿Cuántos eventos hay en los registros?
print('Nombres de los eventos:')
for event in data['EventName'].unique(): print(event)
print()
print('Número de eventos:',len(data['EventName'].unique()))

In [ ]:
# ¿Cuántos usuarios hay en los registros?

print('Número de Usuarios:', len(data['DeviceIDHash'].unique()))

In [ ]:
# ¿Cuál es el promedio de eventos por usuario?


print(f"El promedio de eventos por usuario es de: {data.groupby('DeviceIDHash')['EventName'].count().mean():.2f} eventos")

In [ ]:
 # ¿Qué periodo de tiempo cubren los datos? Encuentra la fecha máxima y mínima. 
print(f"Fecha mínima: {data['daytime'].min()}")
print(f"Fecha máxima: {data['daytime'].max()}")

# Traza un histograma por fecha y hora
plt.figure(figsize=(12, 6))
data['daytime'].hist(bins=100)
plt.xlabel('Fecha y Hora')
plt.ylabel('Frecuencia')
plt.title('Histograma de eventos por fecha')
plt.xticks(rotation=45)
plt.show()

In [ ]:
 # ¿Puedes tener seguridad de que tienes datos igualmente completos para todo el periodo? Los eventos más antiguos podrían terminar en los registros de algunos usuarios por razones técnicas y esto podría sesgar el panorama general. 
 # Encuentra el momento en el que los datos comienzan a estar completos e ignora la sección anterior. 
data_f = data[data['date'] > pd.to_datetime('2019-08-01').date()]
 # ¿Qué periodo representan realmente los datos?
print('Se observa que los datos son mas consistentes y parecen estar mas completos apartir del primero de agosto')
print()
print(data_f.head())


In [ ]:
# Traza un histograma por fecha y hora
plt.figure(figsize=(12, 6))
data_f['daytime'].hist(bins=100)
plt.xlabel('Fecha y Hora')
plt.ylabel('Frecuencia')
plt.title('Histograma de eventos por fecha (filtrado)')
plt.xticks(rotation=45)
plt.show()

Se observa que hay una tendencia a utilizar la app en aproximadamente la misma hora del dia


Paso 4. Estudiar el embudo de eventos

    Observa qué eventos hay en los registros y su frecuencia de suceso. Ordénalos por frecuencia.
    Encuentra la cantidad de usuarios que realizaron cada una de estas acciones. Ordena los eventos por el número de usuarios. Calcula la proporción de usuarios que realizaron la acción al menos una vez.
    ¿En qué orden crees que ocurrieron las acciones? ¿Todas son parte de una sola secuencia? No es necesario tenerlas en cuenta al calcular el embudo.
    Utiliza el embudo de eventos para encontrar la proporción de usuarios que pasan de una etapa a la siguiente. Por ejemplo, para la secuencia de eventos A → B → C, calcula la proporción de usuarios en la etapa B a la cantidad de usuarios en la etapa A y la proporción de usuarios en la etapa C a la cantidad en la etapa B.
    ¿En qué etapa pierdes más usuarios?
    ¿Qué porcentaje de usuarios hace todo el viaje desde su primer evento hasta el pago?


In [ ]:
#  Observa qué eventos hay en los registros y su frecuencia de suceso. Ordénalos por frecuencia.
data_f.groupby('EventName')['daytime'].agg('count').sort_values()

Se observa que el evento mas frecuente es ver la pantalla principal, por el contrario casi ningun usuario realizo el tutorial 

In [ ]:
#Encuentra la cantidad de usuarios que realizaron cada una de estas acciones. Ordena los eventos por el número de usuarios.
eventByUser = data_f.groupby('EventName')['DeviceIDHash'].nunique().sort_values(ascending= False)
# Calcula la proporción de usuarios que realizaron la acción al menos una vez.
n_usuarios = data_f['DeviceIDHash'].nunique()

print('El numero de eventos por usuario es:')
print(eventByUser)
print()
print('la proporcion es:')
print(eventByUser/n_usuarios)


Se observa que menos de la mitad de usuarios que abren la aplicacion concretan la compra 

In [ ]:
# definimos una funcion para uso posterior 

def funnl_ord(data, funnel):
    """ Se genera una funcion para establecer el orden de eventos. 
    Se toman dos argumentos; el origen de los datos y el funnel ordenado de los eventos"""

    funn_eve = pd.DataFrame(columns=['event', 'unique_users', 'proportion'])

    for i in range(len(funnel) - 1):
        current_event = funnel[i]
        next_event = funnel[i + 1]

        current_users = data[data['EventName'] == current_event]['DeviceIDHash'].unique()
        next_users = data[data['EventName'] == next_event]['DeviceIDHash'].unique()

        proportion = len(set(next_users) & set(current_users)) / len(current_users)

        new_row = pd.DataFrame({'event': [next_event], 'unique_users': [len(next_users)], 'proportion': [proportion]})
        funn_eve = pd.concat([funn_eve, new_row], ignore_index=True)
    print(funn_eve)

In [ ]:
#¿En qué orden crees que ocurrieron las acciones? ¿Todas son parte de una sola secuencia? No es necesario tenerlas en cuenta al calcular el embudo.
#Utiliza el embudo de eventos para encontrar la proporción de usuarios que pasan de una etapa a la siguiente. Por ejemplo, para la secuencia de eventos A → B → C, calcula la proporción de usuarios en la etapa B a la cantidad de usuarios en la etapa A y la proporción de usuarios en la etapa C a la cantidad en la etapa B.
#¿En qué etapa pierdes más usuarios?

#Debido a que los usuarios parecen omitir el tutorial no se considera para el embudo de eventos 
#se genera embudo
funnel = ['MainScreenAppear','OffersScreenAppear','CartScreenAppear','PaymentScreenSuccessful']

funnl_ord(data_f,funnel)


Se observa que poco mas de la mitad de usuarios que abren la apliacion pasan a la pantalla de oferta pero el 80% de esos agregan el prducto a su carrito y casi el 100% de los usuarios con producto en el carrito realizan la compra 

In [ ]:
#  ¿Qué porcentaje de usuarios hace todo el viaje desde su primer evento hasta el pago?
def user_full_trip(data, funnel, show=True):
    """ Se genera una funcion para dar el porcentaje de usuarios que recorren el funnel completo
    desde el inicio hasta el final"""

    # Encuentra los usuarios que realizaron el primer evento
    first_event_users = data[data['EventName'] == funnel[0]]['DeviceIDHash'].unique()
    # Encuentra los usuarios que realizaron el último evento (pago)
    last_event_users = data[data['EventName'] == funnel[-1]]['DeviceIDHash'].unique()
    # Calcula el porcentaje de usuarios que realizaron todo el viaje
    percentage_full_journey = len(set(last_event_users) & set(first_event_users)) / len(first_event_users) * 100
    
    if show:
        print(f"El porcentaje de usuarios que hace todo el viaje desde su primer evento hasta el pago es: {percentage_full_journey:.2f}%")
    else:
        return percentage_full_journey


user_full_trip(data_f,funnel)


Paso 5. Estudiar los resultados del experimento

   - ¿Cuántos usuarios hay en cada grupo?
   - Tenemos dos grupos de control en el test A/A, donde comprobamos nuestros mecanismos y cálculos. Observa si hay una diferencia estadísticamente significativa entre las muestras 246 y 247.
   - Selecciona el evento más popular. En cada uno de los grupos de control, encuentra la cantidad de usuarios que realizaron esta acción. Encuentra su proporción. Comprueba si la diferencia entre los grupos es estadísticamente significativa. Repite el procedimiento para todos los demás eventos (ahorrarás tiempo si creas una función especial para esta prueba). ¿Puedes confirmar que los grupos se dividieron correctamente?
   - Haz lo mismo para el grupo con fuentes alteradas. Compara los resultados con los de cada uno de los grupos de control para cada evento de forma aislada. Compara los resultados con los resultados combinados de los grupos de control. ¿Qué conclusiones puedes sacar del experimento?
   - ¿Qué nivel de significancia has establecido para probar las hipótesis estadísticas mencionadas anteriormente? Calcula cuántas pruebas de hipótesis estadísticas has realizado. Con un nivel de significancia estadística de 0.1, uno de cada 10 resultados podría ser falso. ¿Cuál debería ser el nivel de significancia? Si deseas cambiarlo, vuelve a ejecutar los pasos anteriores y comprueba tus conclusiones.

In [ ]:
usuarios_por_grupo = data_f.pivot_table(
    index='ExpId', values='DeviceIDHash', aggfunc='nunique')

usuarios_por_grupo

In [ ]:
gpo_246 = data_f[data_f['ExpId'] == 246]['DeviceIDHash'].unique()
gpo_247 = data_f[data_f['ExpId'] == 247]['DeviceIDHash'].unique()

In [ ]:
t_stat, p_value = stats.ttest_ind(gpo_246, gpo_247, equal_var=False)
print(f"Estadístico t: {t_stat:.4f}, Valor p: {p_value:.4f}")

Se observa que la significancia estadistica es muy alta por lo tanto las diferencias no existen entre los grupos. Esto es algo esperado ya que ambas muestras son del grupo control.


In [ ]:
#ver evento mas popular
data_f.pivot_table(index='EventName', columns='ExpId', aggfunc='nunique', values='DeviceIDHash')

Se encuentra que el evento mas popular es la presenciation de la pantalla principal

In [ ]:
# Se genera una funcion para el calulo de proporciones (se experimenta con PEP 3107)
def u_per_ev_gpo(data: pd.DataFrame, gpo: int, evento: str) -> None:
    """Se genera funcion para calcular la proporcion del evento"""
    u_eve = data[(data['ExpId'] == gpo) & (data['EventName'] == evento)]['DeviceIDHash'].nunique()
    u_gpo = data[data['ExpId'] == gpo]['DeviceIDHash'].nunique()

    Prop = u_eve / u_gpo
    print(f'La proporcion de usuarios del grupo {gpo}, que realizaron el evento es: {Prop:.2%}')

In [ ]:
for gpo in [246, 247, 248]:
    u_per_ev_gpo(data_f, gpo, 'MainScreenAppear')


In [ ]:
def t_dif_gpo(gpo1: pd.DataFrame, gpo2: pd.DataFrame, P: float=0.05) -> None:
    t_stat_evento, p_value_evento = stats.ttest_ind(
        gpo1, gpo2, equal_var=False)
    print('El resultado de la prueba es:')
    print(f"Estadístico t: {t_stat_evento:.4F}, Valor p: {p_value_evento:.4F}")

    if p_value_evento < P:
        print("Existen diferencias significativas.")
    else:
        print("No hay diferencias significativas.")


In [ ]:
gpo_246 = data[(data['ExpId'] == 246) & (data['EventName'] == 'MainScreenAppear')]['DeviceIDHash'].value_counts()
gpo_247 = data[(data['ExpId'] == 247) & (data['EventName'] == 'MainScreenAppear')]['DeviceIDHash'].value_counts()
gpo_248 = data[(data['ExpId'] == 248) & (data['EventName'] == 'MainScreenAppear')]['DeviceIDHash'].value_counts()
gpo_ctrl = data[(data['ExpId'] != 248) & (data['EventName'] == 'MainScreenAppear')]['DeviceIDHash'].value_counts()

In [ ]:
t_dif_gpo(gpo_246,gpo_247)
print('No hay diferencia entre los grupos control, cosa que se espera :)')


In [ ]:
t_dif_gpo(gpo_ctrl,gpo_248)
print('No hay diferencia entre los grupos control y experimental \nEsto denota que nuestro experimento no tuvo efectividad')

In [ ]:
#Se repiten los analisis con la nueva p sugerida

t_dif_gpo(gpo_246,gpo_247,0.1)


In [ ]:
t_dif_gpo(gpo_ctrl, gpo_248,0.1)

Se observa que los resultados de la prueba ya son significativos al ajustar la significancia, sin embargo; el aumentar la significancia para hacer que los datos se amolden a la hipotesis es una tecnica que no debe ser realizada, ya que la significancia debe establecerce a priori para garantizar la confiabilidad de las pruebas A/B.